In [112]:
#import unittest
#from unittest import TestCase
import pandas as pd
import os, sys
from io import StringIO
import pkg_resources
import importlib
from colorama import Fore, Back, Style 

In [113]:
import devanalyst.simulation.GenerateTimecards as timecards
from devanalyst.simulation.GenerateTimecards import UserStoriesRepo, UserStory, UserStoryStatus, Ticket, TicketsRepo

In [114]:
# Reloads the devanalyst modules used in this notebook. Useful to refresh after code changes in the devanalyst modules
def reload():
    global timecards
    timecards = importlib.import_module('devanalyst.simulation.GenerateTimecards')
    timecards = importlib.reload(timecards)

In [115]:
# Re-run this each time the source code changes, so that we have the latest version
#reload()

In [116]:
#globals used in this test
PM_DF = None
DEV_DF = None
TEAMS_DF = None

In [117]:
def loadTestResources():
    global PM_DF
    global DEV_DF
    resource_path = '/'.join(('Resources', 'Simulation', 'Team.xlsx'))
    resource_package = 'devanalyst'
    fullpath = pkg_resources.resource_filename(resource_package, resource_path)
    PM_DF = pd.read_excel(fullpath, 'PMs')
    DEV_DF = pd.read_excel(fullpath, 'Dev')

<h1>Test User Story Status</h1>

In [118]:
# Helper methods to the test
def format_ticket(ticket):
    return ('Ticket:' + ticket.ticketId \
            + ',\n\t\t storyId=' + ticket.userStoryId \
            + ',\n\t\t costToFix=' + str(ticket.costToFix) \
            + ',\n\t\t effortToDate=' + str(ticket.effortToDate) \
            + ',\n\t\t percentAchieved=' + str(ticket.percentAchieved))
                
def format_tickets(tickets):
    output = ''
    for ticket in tickets:
        output = output + '\n\t\t{' + format_ticket(ticket) + '}'
    return output
            
def format_uss(uss):
    return ('\n *** USS:' + uss.userStoryId \
            + '\n\t achieved=' + str(uss.percentAchieved) \
            + ',\n\t planned=' + str(uss.planned) \
            + ',\n\t sprintPlanned=' + str(uss.sprintPlanned) \
            + ',\n\t tickets=' + format_tickets(uss.pendingTickets))


def format_item(item, item_label, sprint, timeInSprint):
    return ('\n *** ' + item_label + ' at ' + timeInSprint + ' of sprint' + str(sprint) + ': ' \
            + '\n\t userStoryId=' + str(item.userStoryId) \
            + ',\n\t taskType=' + str(item.taskType) \
            + ',\n\t ticketId=' + str(item.ticketId) \
            + ',\n\t estimate=' + '{0:.2f}'.format(item.estimate) \
            + ',\n\t percentAchieved=' + str(item.percentAchieved)  \
            + ',\n\t sprintPlanned=' + str(item.sprintPlanned))
 
def test_uss():
    output = '' 
    repo = UserStoriesRepo([UserStory('Story A', 25, 'Joe Developer', 'Amy PM'), \
                          UserStory('Story B', 17, 'Alex Developer', 'Kate PM')])
    uss = UserStoryStatus('Story B', 0.0)
    uss.planned = True
    uss.sprintPlanned = 1
    output = output + (format_uss(uss))
    item = uss.generateWorkItems(repo)[0]
    output = output + (format_item(item, 'Item#1', 1, 'start'))
    item.percentAchieved = 0.7
    newTickets = [Ticket('Bug 100','Story B', 4), Ticket('Bug 101','Story B', 1.5)]
    bugRepo = TicketsRepo(newTickets)
    output = output + (format_item(item, 'Item#1', 1, 'end'))
    uss.updateStatus([item], newTickets, bugRepo)
    uss.sprintPlanned = 2
    output = output + (format_uss(uss))

    items = uss.generateWorkItems(repo)
    item=items[0]
    output = output + (format_item(item, 'Item#1', 2, 'start'))
    item=items[1]
    output = output + (format_item(item, 'Item#2', 2, 'start'))
    item=items[2]
    output = output + (format_item(item, 'Item#3', 2, 'start'))
    items[0].percentAchieved = 0.9
    items[1].percentAchieved = 1.0
    items[2].percentAchieved = 0.5
    item=items[0]
    item=items[0]
    item=items[1]
    output = output + (format_item(item, 'Item#2', 2, 'end'))
    item=items[2]
    output = output + (format_item(item, 'Item#3', 2, 'end'))
    uss.updateStatus(items, [], bugRepo)
    output = output + (format_uss(uss))
    return output

In [119]:
# Expected output
test_uss_EXPECTED = '\n *** USS:Story B\n\t achieved=0.0,\n\t planned=True,\n\t sprintPlanned=1,\n\t tickets=\n *** Item#1 at start of sprint1: \n\t userStoryId=Story B,\n\t taskType=UNFINISHED_STORIES,\n\t ticketId=None,\n\t estimate=17.00,\n\t percentAchieved=0.0,\n\t sprintPlanned=1\n *** Item#1 at end of sprint1: \n\t userStoryId=Story B,\n\t taskType=UNFINISHED_STORIES,\n\t ticketId=None,\n\t estimate=17.00,\n\t percentAchieved=0.7,\n\t sprintPlanned=1\n *** USS:Story B\n\t achieved=0.7,\n\t planned=True,\n\t sprintPlanned=2,\n\t tickets=\n\t\t{Ticket:Bug 100,\n\t\t storyId=Story B,\n\t\t costToFix=4,\n\t\t effortToDate=0.0,\n\t\t percentAchieved=0.0}\n\t\t{Ticket:Bug 101,\n\t\t storyId=Story B,\n\t\t costToFix=1.5,\n\t\t effortToDate=0.0,\n\t\t percentAchieved=0.0}\n *** Item#1 at start of sprint2: \n\t userStoryId=Story B,\n\t taskType=UNFINISHED_STORIES,\n\t ticketId=None,\n\t estimate=5.10,\n\t percentAchieved=0.0,\n\t sprintPlanned=2\n *** Item#2 at start of sprint2: \n\t userStoryId=Story B,\n\t taskType=BUGS_ON_UNFINISHED_STORIES,\n\t ticketId=Bug 100,\n\t estimate=2.00,\n\t percentAchieved=0.0,\n\t sprintPlanned=2\n *** Item#3 at start of sprint2: \n\t userStoryId=Story B,\n\t taskType=BUGS_ON_UNFINISHED_STORIES,\n\t ticketId=Bug 101,\n\t estimate=2.00,\n\t percentAchieved=0.0,\n\t sprintPlanned=2\n *** Item#2 at end of sprint2: \n\t userStoryId=Story B,\n\t taskType=BUGS_ON_UNFINISHED_STORIES,\n\t ticketId=Bug 100,\n\t estimate=2.00,\n\t percentAchieved=1.0,\n\t sprintPlanned=2\n *** Item#3 at end of sprint2: \n\t userStoryId=Story B,\n\t taskType=BUGS_ON_UNFINISHED_STORIES,\n\t ticketId=Bug 101,\n\t estimate=2.00,\n\t percentAchieved=0.5,\n\t sprintPlanned=2\n *** USS:Story B\n\t achieved=0.97,\n\t planned=True,\n\t sprintPlanned=2,\n\t tickets=\n\t\t{Ticket:Bug 101,\n\t\t storyId=Story B,\n\t\t costToFix=1.5,\n\t\t effortToDate=2.0,\n\t\t percentAchieved=0.5}'

In [120]:
test_uss_ACTUAL = test_uss()

In [121]:
# Uncomment to update test_uss_EXPECTED with the value of test_uss_ACTUAL by copy-and-paste
#test_uss_ACTUAL

In [122]:
print(Back.BLUE + Fore.WHITE + '--------------------- ACTUAL -------------------------', \
      Back.RESET + Fore.BLUE + '\n' + test_uss_ACTUAL) 

--------------------- ACTUAL ------------------------- 

 *** USS:Story B
	 achieved=0.0,
	 planned=True,
	 sprintPlanned=1,
	 tickets=
 *** Item#1 at start of sprint1: 
	 userStoryId=Story B,
	 taskType=UNFINISHED_STORIES,
	 ticketId=None,
	 estimate=17.00,
	 percentAchieved=0.0,
	 sprintPlanned=1
 *** Item#1 at end of sprint1: 
	 userStoryId=Story B,
	 taskType=UNFINISHED_STORIES,
	 ticketId=None,
	 estimate=17.00,
	 percentAchieved=0.7,
	 sprintPlanned=1
 *** USS:Story B
	 achieved=0.7,
	 planned=True,
	 sprintPlanned=2,
	 tickets=
		{Ticket:Bug 100,
		 storyId=Story B,
		 costToFix=4,
		 effortToDate=0.0,
		 percentAchieved=0.0}
		{Ticket:Bug 101,
		 storyId=Story B,
		 costToFix=1.5,
		 effortToDate=0.0,
		 percentAchieved=0.0}
 *** Item#1 at start of sprint2: 
	 userStoryId=Story B,
	 taskType=UNFINISHED_STORIES,
	 ticketId=None,
	 estimate=5.10,
	 percentAchieved=0.0,
	 sprintPlanned=2
 *** Item#2 at start of sprint2: 
	 userStoryId=Story B,
	 taskType=BUGS_ON_UNFINISHED_STORIES

In [123]:
print(Back.GREEN + Fore.WHITE + '--------------------- EXPECTED -----------------------', \
      Back.RESET + Fore.GREEN + '\n' + test_uss_EXPECTED) 

--------------------- EXPECTED ----------------------- 

 *** USS:Story B
	 achieved=0.0,
	 planned=True,
	 sprintPlanned=1,
	 tickets=
 *** Item#1 at start of sprint1: 
	 userStoryId=Story B,
	 taskType=UNFINISHED_STORIES,
	 ticketId=None,
	 estimate=17.00,
	 percentAchieved=0.0,
	 sprintPlanned=1
 *** Item#1 at end of sprint1: 
	 userStoryId=Story B,
	 taskType=UNFINISHED_STORIES,
	 ticketId=None,
	 estimate=17.00,
	 percentAchieved=0.7,
	 sprintPlanned=1
 *** USS:Story B
	 achieved=0.7,
	 planned=True,
	 sprintPlanned=2,
	 tickets=
		{Ticket:Bug 100,
		 storyId=Story B,
		 costToFix=4,
		 effortToDate=0.0,
		 percentAchieved=0.0}
		{Ticket:Bug 101,
		 storyId=Story B,
		 costToFix=1.5,
		 effortToDate=0.0,
		 percentAchieved=0.0}
 *** Item#1 at start of sprint2: 
	 userStoryId=Story B,
	 taskType=UNFINISHED_STORIES,
	 ticketId=None,
	 estimate=5.10,
	 percentAchieved=0.0,
	 sprintPlanned=2
 *** Item#2 at start of sprint2: 
	 userStoryId=Story B,
	 taskType=BUGS_ON_UNFINISHED_STORIES

In [124]:
#class TestUSS(TestCase):
#
#    def test(self):
#        self.assertTrue(test_uss_ACTUAL == test_uss_EXPECTED)

In [125]:
#TestUSS().test()

In [126]:
#unittest.main(argv=[''], verbosity=2, exit=False)